In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,AveragePooling2D
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

In [ ]:
unzip_data("nlp_getting_started.zip")

In [ ]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
# The test data doesn't have a target (that's what we'd try to predict)
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
print(f'Total training samples: {len(train_df)}')
print(f'Total test samples {len(test_df)}')
print(f'Total samples: {len(train_df) + len(test_df)}')

Total training samples: 7613
Total test samples 3263
Total samples: 10876


# Visualizing a text dataset

In [ ]:
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
@nataliealund 
Parents of Colorado theater shooting victim fear copycat massacre

http://t.co/LvlH3W3aWO
#Antioch

http://t.co/vIwXY1XDYK

---

Target: 1 (real disaster)
Text:
Earthquake and tsunami that occurred in Japan 'free speech' is also swallowed. http://t.co/TJyyFT6NV0

---

Target: 0 (not real disaster)
Text:
I can't listen to Darude Sandstorm without expecting airhorns now

---

Target: 1 (real disaster)
Text:
#Nuclear policy of #Japan without responsibility about Nuclear #Disaster will repeat same #failure.
#annonymous #guardian #NYTimes #Reuters

---

Target: 1 (real disaster)
Text:
Warfighting Robots Could Reduce Civilian Casualties So Calling for a Ban Now Is Premature http://t.co/lzff4pT4AZ #FTSN #FTSNNewsdesk #Û_

---



#Split Data

In [ ]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility

In [ ]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

#Converting text into numbers

## Text vectorization (tokenization)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [ ]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [ ]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [ ]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
No #news of #hostages in #Libya

http://t.co/bjjOIfzUhL

#India #terrorism #Africa #AP #TS #NRI #News #TRS #TDP #BJP http://t.co/IywZAlLsN4      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  40,   58,    6,  422,    4, 1009,    1,  544,  361, 1332, 1556,
        3305, 3584,   58, 3307]])>

In [ ]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
Fire in Pisgah National Forest grows to 375 acres http://t.co/d7zxZ42QW1      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  42,    4, 3534,  372,  188, 1403,    5, 4178, 1722,    1,    0,
           0,    0,    0,    0]])>

In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


#Creating an Embedding using an Embedding Layer

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                              output_dim = 128,
                              embeddings_initializer = 'uniform',
                              input_length = max_length,
                              name = 'embedding_1')

embedding

In [ ]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
'Mages of Fairy Tail.. Specialize in property damage!' - Natsu Dragneel      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.0446792 , -0.0327296 ,  0.01944308, ...,  0.00552434,
          0.02740857, -0.00306237],
        [-0.04938959, -0.01527452,  0.04749585, ...,  0.00914271,
          0.00259786, -0.03235149],
        [-0.01805013,  0.00396691, -0.00768809, ..., -0.01371616,
         -0.03086761,  0.04235205],
        ...,
        [ 0.03741227,  0.00934342, -0.02519831, ...,  0.04730524,
          0.0021214 ,  0.0350779 ],
        [ 0.03741227,  0.00934342, -0.02519831, ...,  0.04730524,
          0.0021214 ,  0.0350779 ],
        [ 0.03741227,  0.00934342, -0.02519831, ...,  0.04730524,
          0.0021214 ,  0.0350779 ]]], dtype=float32)>

In [ ]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-4.46792021e-02, -3.27296034e-02,  1.94430836e-02, -3.28644291e-02,
       -2.40587126e-02, -4.96476069e-02, -2.19652783e-02, -3.76912467e-02,
        3.88805903e-02, -2.61290073e-02, -1.70740969e-02, -3.65284570e-02,
        4.60623242e-02, -1.31931528e-02,  9.88006592e-04, -2.96551231e-02,
        4.25102375e-02,  1.49581470e-02,  2.69008540e-02, -3.54364999e-02,
       -2.95033939e-02,  1.98610909e-02,  5.10716438e-03,  1.74718983e-02,
       -1.79693475e-02,  1.69995762e-02,  2.01028921e-02,  3.52770090e-03,
       -1.75235048e-02, -4.88548651e-02,  4.43246029e-02, -3.23703289e-02,
        1.47292875e-02,  2.76310556e-02, -5.80042601e-03,  1.60524994e-03,
       -2.49526855e-02,  1.02864578e-03, -4.06645425e-02, -4.39458378e-02,
       -4.87689860e-02, -5.87539747e-03,  4.75113504e-02, -3.27123292e-02,
       -3.07105538e-02,  2.31767334e-02,  1.09895319e-03,  1.97101384e-04,
       -3.63717228e-03,  4.69786786e-02, -3.99715900

# Model 0: Naive Bayes

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:

baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [ ]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = "weighted")
  model_results = {'accuracy': model_accuracy,
             'precision': model_precision,
             'recall': model_recall,
             'f1': model_f1}

  return model_results

In [ ]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

# # Model 1: A simple dense model

In [ ]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs= layers.Dense(1, activation='sigmoid')(x)
model_1 = tf.keras.Model(inputs, outputs, name = 'model_1_dense')

model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [ ]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [ ]:
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 5s 20ms/step - loss: 0.6106 - accuracy: 0.6917 - val_loss: 0.5364 - val_accuracy: 0.7507
Epoch 2/5
215/215 [==============================] - 5s 23ms/step - loss: 0.4418 - accuracy: 0.8189 - val_loss: 0.4689 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 4s 20ms/step - loss: 0.3468 - accuracy: 0.8590 - val_loss: 0.4586 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 4s 19ms/step - loss: 0.2852 - accuracy: 0.8917 - val_loss: 0.4636 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 5s 24ms/step - loss: 0.2383 - accuracy: 0.9121 - val_loss: 0.4761 - val_accuracy: 0.7874


In [ ]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 10ms/step - loss: 0.4761 - accuracy: 0.7874


[0.4760918915271759, 0.787401556968689]

# Model 2 using LSTM

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout ,AveragePooling1D
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers
model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")
# Create LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)
x = layers.Dropout(0.2)(x)
x = layers.LSTM(32)(x)
print(x.shape)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dense(16, activation="relu")(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2")

NameError: name 'max_vocab_length' is not defined

In [ ]:
# Compile model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_2.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 dropout (Dropout)           (None, 15, 128)           0         
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                           

In [ ]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 12s 37ms/step - loss: 0.5882 - accuracy: 0.6911 - val_loss: 0.4723 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 9s 43ms/step - loss: 0.4078 - accuracy: 0.8533 - val_loss: 0.4952 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 7s 34ms/step - loss: 0.3145 - accuracy: 0.9022 - val_loss: 0.5919 - val_accuracy: 0.7769
Epoch 4/5
215/215 [==============================] - 9s 42ms/step - loss: 0.2521 - accuracy: 0.9267 - val_loss: 0.6279 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 9s 43ms/step - loss: 0.2050 - accuracy: 0.9419 - val_loss: 0.6886 - val_accuracy: 0.7717


In [ ]:
model_2.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 8ms/step - loss: 0.6886 - accuracy: 0.7717


[0.6885508298873901, 0.7716535329818726]

# Model 3:RNN with GRU

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3")

In [ ]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_3.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                           

In [ ]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 17s 46ms/step - loss: 0.5277 - accuracy: 0.7306 - val_loss: 0.4546 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 7s 31ms/step - loss: 0.3204 - accuracy: 0.8682 - val_loss: 0.4994 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 10s 48ms/step - loss: 0.2168 - accuracy: 0.9187 - val_loss: 0.5612 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 9s 41ms/step - loss: 0.1519 - accuracy: 0.9463 - val_loss: 0.6494 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 10s 47ms/step - loss: 0.1141 - accuracy: 0.9587 - val_loss: 0.6485 - val_accuracy: 0.7717


In [ ]:
model_3.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 5ms/step - loss: 0.6485 - accuracy: 0.7717


[0.648526132106781, 0.7716535329818726]

#Model 4:1D Conv layers Model

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers
model_4_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_4")

inputs = layers.Input([1,], dtype="string")

x = text_vectorizer(inputs)
x = model_4_embedding(x)

x = layers.Conv1D(64, 8, activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x)

x = layers.GlobalAveragePooling1D()(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model_4 = tf.keras.Model(inputs, outputs)

In [ ]:
model_4.compile(loss="binary_crossentropy",
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=["accuracy"])

In [ ]:
model_4.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_4 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d (Conv1D)             (None, 15, 64)            65600     
                                                                 
 batch_normalization (Batch  (None, 15, 64)            256       
 Normalization)                                                  
                                                                 
 global_average_pooling1d_1  (None, 64)                0     

In [ ]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 9s 31ms/step - loss: 0.5019 - accuracy: 0.7561 - val_loss: 0.6159 - val_accuracy: 0.7415
Epoch 2/5
215/215 [==============================] - 6s 26ms/step - loss: 0.2922 - accuracy: 0.8794 - val_loss: 0.5021 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 5s 23ms/step - loss: 0.1896 - accuracy: 0.9267 - val_loss: 0.5473 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 6s 30ms/step - loss: 0.1313 - accuracy: 0.9542 - val_loss: 0.7551 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0904 - accuracy: 0.9676 - val_loss: 0.7789 - val_accuracy: 0.7546


In [ ]:
model_4.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.7789 - accuracy: 0.7546


[0.7788501381874084, 0.7545931935310364]

# Model 4:Transfer learning model

In [ ]:
import tensorflow_hub as hub
sentence_encoder_layer = hub.KerasLayer("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2",
                                        input_shape=[],
                                        dtype=tf.string,
                                        name="USE")

print(sentence_encoder_layer)

tf.Tensor(
[[-0.00945472 -0.06090169 -0.02792026 ... -0.01716043 -0.03901615
   0.03263445]
 [ 0.0321504  -0.03578445  0.0510855  ... -0.01297739 -0.01233434
   0.04015184]], shape=(2, 512), dtype=float32)


In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers
model_5_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_4")
model_5 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(128, activation="relu"),
layers.Dense(64,activation='relu'),
layers.Dense(32,activation='relu'),

    layers.Dense(1, activation="sigmoid")
])

model_5.compile(loss="binary_crossentropy",
                       optimizer=tf.keras.optimizers.Adam(),
                       metrics=["accuracy"])

model_5.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_16 (Dense)            (None, 128)               65664     
                                                                 
 dense_17 (Dense)            (None, 64)                8256      
                                                                 
 dense_18 (Dense)            (None, 32)                2080      
                                                                 
 dense_19 (Dense)            (None, 1)                 33        
                                                                 
Total params: 256873857 (979.90 MB)
Trainable params: 76033 (297.00 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [ ]:
sentence_encoder_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 9s 22ms/step - loss: 0.4667 - accuracy: 0.7921 - val_loss: 0.4309 - val_accuracy: 0.8071
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3903 - accuracy: 0.8289 - val_loss: 0.4196 - val_accuracy: 0.8123
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3565 - accuracy: 0.8488 - val_loss: 0.4162 - val_accuracy: 0.8255
Epoch 4/5
215/215 [==============================] - 4s 20ms/step - loss: 0.3121 - accuracy: 0.8733 - val_loss: 0.4274 - val_accuracy: 0.8189
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.2510 - accuracy: 0.9041 - val_loss: 0.4564 - val_accuracy: 0.8228
